# Field4D 可視化チュートリアル

このノートブックでは、`Field4D` クラスの描画・解析機能を実践します。
コア機能については `intro_Field4D.ipynb` を参照してください。

## 学習内容
1. 2D断面マップ（`plot_map2d`）
2. 多点時系列の抽出と重ね書き（`plot_timeseries_points`）
3. 1D分布プロファイル（`plot_profile`）
4. 応用解析（`diff`, `zscore`, `time_stat_map`, `time_space_map`）


## セットアップ


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from astropy import units as u

from gwexpy.types import Field4D

# 再現性のためのシード設定
rng = np.random.default_rng(seed=42)

# Matplotlib スタイル
plt.style.use('default')
%matplotlib inline

## 共通テストデータの生成

再現性と検証を容易にするため、**解析解を持つデータ**を使用します。
ここでは「移動するガウシアン」を生成します。


In [ ]:
# 軸定義（等間隔・単位付き）
nt, nx, ny, nz = 20, 32, 32, 1  # nz=1 は2D空間として扱う
t = np.linspace(0, 1, nt) * u.s
x = np.linspace(-5, 5, nx) * u.m
y = np.linspace(-5, 5, ny) * u.m
z = np.array([0]) * u.m  # 2D case

# メッシュグリッド
T, X, Y = np.meshgrid(t.value, x.value, y.value, indexing='ij')

# 移動ガウシアン: 中心が時間とともにx方向に移動
x_center = T * 5  # 5 m/s で移動
sigma = 1.0  # ガウスの幅
amplitude = 10.0  # 振幅

data = amplitude * np.exp(-((X - x_center)**2 + Y**2) / (2 * sigma**2))
data = data[:, :, :, np.newaxis]  # (t, x, y, z=1)

# Field4D オブジェクトの作成
field = Field4D(
    data,
    unit=u.V,
    axis0=t,
    axis1=x,
    axis2=y,
    axis3=z,
    axis_names=['t', 'x', 'y', 'z'],
    axis0_domain='time',
    space_domain='real'
)

print(f'Shape: {field.shape}')
print(f'Unit: {field.unit}')
print(f'Time range: {t[0]} - {t[-1]}')

## 1. 2D断面マップ（`plot_map2d`）

4Dフィールドから特定の時刻・位置でスライスし、2Dヒートマップを描画します。


In [ ]:
# t=0.5s における XY 断面
fig, ax = field.plot_map2d('xy', at={'t': 0.5 * u.s}, add_colorbar=True)
ax.set_title('XY断面 (t=0.5s)')
plt.tight_layout()
plt.show()

In [ ]:
# 複数時刻の断面を比較
fig, axes = plt.subplots(1, 4, figsize=(14, 3))

for i, t_val in enumerate([0.0, 0.33, 0.66, 1.0]):
    _, ax = field.plot_map2d('xy', at={'t': t_val * u.s}, ax=axes[i], add_colorbar=False)
    ax.set_title(f't = {t_val:.2f} s')

fig.colorbar(axes[0].collections[0], ax=axes, orientation='vertical', label='Amplitude [V]')
plt.tight_layout()
plt.show()

## 2. 多点時系列の抽出と重ね書き

複数の空間点における時系列を抽出し、`plot_timeseries_points` で重ねて描画します。


In [ ]:
# 複数点を定義
points = [
    (0.0 * u.m, 0.0 * u.m, 0.0 * u.m),   # 中心
    (2.5 * u.m, 0.0 * u.m, 0.0 * u.m),  # 右側
    (-2.5 * u.m, 0.0 * u.m, 0.0 * u.m), # 左側
]

fig, ax = field.plot_timeseries_points(points)
ax.set_title('複数点の時系列')
ax.legend(loc='upper right')
plt.tight_layout()
plt.show()

## 3. 1D分布プロファイル

指定した軸に沿った断面プロファイルを `plot_profile` で描画します。


In [ ]:
# t=0.5s, y=0 における X軸方向プロファイル
fig, ax = field.plot_profile('x', at={'t': 0.5 * u.s, 'y': 0.0 * u.m, 'z': 0.0 * u.m})
ax.set_title('X軸プロファイル (t=0.5s, y=0)')
plt.tight_layout()
plt.show()

In [ ]:
# 時刻ごとのプロファイル重ね書き
fig, ax = plt.subplots(figsize=(8, 4))

for t_val in [0.0, 0.25, 0.5, 0.75, 1.0]:
    axis_index, profile_data = field.extract_profile('x', at={'t': t_val * u.s, 'y': 0.0 * u.m, 'z': 0.0 * u.m})
    ax.plot(axis_index.value, profile_data.squeeze(), label=f't={t_val:.2f}s')

ax.set_xlabel('x [m]')
ax.set_ylabel('Amplitude [V]')
ax.set_title('時刻ごとのX軸プロファイル')
ax.legend()
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 4. 応用解析機能

`Field4D` が持つ解析メソッドを使って、より高度な可視化を行います。


### 4.1 差分と比率（`diff`）


In [ ]:
# 異なる2時刻のフィールドをスライスして差分を計算
slice_early = field.slice_map2d('xy', at={'t': 0.2 * u.s})
slice_late = field.slice_map2d('xy', at={'t': 0.8 * u.s})

diff_field = slice_late.diff(slice_early, mode='diff')

fig, axes = plt.subplots(1, 3, figsize=(12, 3))

slice_early.plot_map2d('xy', ax=axes[0], add_colorbar=True)
axes[0].set_title('t=0.2s')

slice_late.plot_map2d('xy', ax=axes[1], add_colorbar=True)
axes[1].set_title('t=0.8s')

diff_field.plot_map2d('xy', ax=axes[2], add_colorbar=True, cmap='RdBu_r')
axes[2].set_title('差分 (0.8s - 0.2s)')

plt.tight_layout()
plt.show()

### 4.2 Z-score 正規化


In [ ]:
# 全時間軸を使ってz-scoreを計算
field_zscore = field.zscore()

fig, axes = plt.subplots(1, 2, figsize=(10, 4))

field.plot_map2d('xy', at={'t': 0.5 * u.s}, ax=axes[0], add_colorbar=True)
axes[0].set_title('元データ')

field_zscore.plot_map2d('xy', at={'t': 0.5 * u.s}, ax=axes[1], add_colorbar=True, cmap='coolwarm')
axes[1].set_title('Z-score')

plt.tight_layout()
plt.show()

print(f'Z-score unit: {field_zscore.unit}')

### 4.3 時間要約マップ（`time_stat_map`）


In [ ]:
# 時間方向の統計量を計算
mean_map = field.time_stat_map(stat='mean')
std_map = field.time_stat_map(stat='std')
max_map = field.time_stat_map(stat='max')

fig, axes = plt.subplots(1, 3, figsize=(12, 3))

mean_map.plot_map2d('xy', ax=axes[0], add_colorbar=True)
axes[0].set_title('時間平均')

std_map.plot_map2d('xy', ax=axes[1], add_colorbar=True)
axes[1].set_title('時間標準偏差')

max_map.plot_map2d('xy', ax=axes[2], add_colorbar=True)
axes[2].set_title('時間最大値')

plt.tight_layout()
plt.show()

### 4.4 時間-空間マップ（`plot_time_space_map`）

時間軸と1つの空間軸を使った2Dマップ（ストリークカメラ的な表示）です。


In [ ]:
# y=0 における時間-x マップ
fig, ax = field.plot_time_space_map('x', at={'y': 0.0 * u.m}, add_colorbar=True)
ax.set_title('時間-X マップ (y=0)')
plt.tight_layout()
plt.show()

## まとめ

このノートブックでは、`Field4D` の可視化機能を学びました：

1. **2D断面マップ**: `plot_map2d` で任意の平面を可視化
2. **時系列抽出**: `extract_points` + `plot_timeseries_points` で複数点を比較
3. **1Dプロファイル**: `extract_profile` + `plot_profile` で断面分布を確認
4. **応用解析**: `diff`, `zscore`, `time_stat_map`, `time_space_map` による高度な解析

これらを組み合わせることで、4次元時空間データの包括的な可視化が可能です。
